In [1]:
!pip install transformers
!pip install torch torchvision
!pip install pandas
!pip install scikit-learn
!pip install sentencepiece
!pip install sacremoses

/Users/jayjoshi/.zshenv:3: unmatched "
/Users/jayjoshi/.zshenv:3: unmatched "
/Users/jayjoshi/.zshenv:3: unmatched "
/Users/jayjoshi/.zshenv:3: unmatched "
/Users/jayjoshi/.zshenv:3: unmatched "
/Users/jayjoshi/.zshenv:3: unmatched "


In [2]:
with open("Finetuned_Models/Spanish/Haternet/labeled_corpus_6K.txt") as f:
    lines = f.readlines()
    
ds = [[y, int(z[1:2])] for _, y, z in [line.split("-,-") for line in lines][1:]]

texts = [i[0] for i in ds]
labels = [i[1] for i in ds]

In [15]:
import pandas as pd

ds_train = pd.read_csv("Finetuned_Models/English/Hateval/hateval2019_en_train.csv")
ds_test = pd.read_csv("Finetuned_Models/English/Hateval/hateval2019_en_test.csv")

x = ds_train['HS']
train_labels = [i for i in x]
train_texts = [i for i in ds_train['text']]

test_labels = [i for i in ds_test['HS']]
test_texts = [i for i in ds_test['text']]


int

In [3]:
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=.1)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.1)


In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('xlm-mlm-xnli15-1024', download_mode="force_redownload")

In [5]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [6]:
import torch

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

In [19]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

model = AutoModelForSequenceClassification.from_pretrained("xlm-mlm-xnli15-1024")

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()

Downloading:   0%|          | 0.00/1.39G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-mlm-xnli15-1024 were not used when initializing XLMForSequenceClassification: ['pred_layer.proj.weight', 'pred_layer.proj.bias']
- This IS expected if you are initializing XLMForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMForSequenceClassification were not initialized from the model checkpoint at xlm-mlm-xnli15-1024 and are newly initialized: ['sequence_summary.summary.weight', 'transformer.position_ids', 'sequence_summary.summary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

Step,Training Loss
10,2.873500
20,2.196900
30,1.814500
40,1.652200
50,1.292000
60,1.708000
70,1.511500
80,2.668300
90,1.669300
100,1.565800


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=912, training_loss=0.9367144135268111, metrics={'train_runtime': 5787.2706, 'train_samples_per_second': 2.519, 'train_steps_per_second': 0.158, 'total_flos': 1599698618676972.0, 'train_loss': 0.9367144135268111, 'epoch': 3.0})